In [1]:
import sys
sys.path
sys.path.append('../tests/')

In [2]:
from tasks import get_linear_task
from nngeometry.generator.jacobian import Jacobian
from nngeometry.object.PMat import PMatDense, PMatDiag, PMatKFAC
from nngeometry.object.vector import random_pvector
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

from tasks import get_fullyconnect_task

In [3]:
loader, lc, parameters, model, function, n_output = get_fullyconnect_task()

In [4]:
from nngeometry.metrics import FIM

In [5]:
dw = random_pvector(layer_collection=lc, device='cuda')

In [6]:
lc.layers

OrderedDict([('net.0.Linear(in_features=784, out_features=15, bias=True)',
              <nngeometry.layercollection.LinearLayer at 0x7f93c6e46550>),
             ('net.2.Linear(in_features=15, out_features=15, bias=True)',
              <nngeometry.layercollection.LinearLayer at 0x7f93c6e46630>),
             ('net.4.Linear(in_features=15, out_features=10, bias=True)',
              <nngeometry.layercollection.LinearLayer at 0x7f93c6e466d8>)])

# Dense matrix

In [7]:
GGN_dense = FIM(layer_collection=lc,
                model=model,
                loader=loader,
                representation=PMatDense,
                n_output=10,
                variant='classif_logits',
                device='cuda')

GGN_dense_inverse = GGN_dense.inverse()

diag_GGN_dense_inverse = torch.diag(GGN_dense_inverse.get_dense_tensor())
print(diag_GGN_dense_inverse.size())

GGNinv_dense_dw = GGN_dense_inverse.mv(dw)

torch.Size([12175])


# Diag matrix

In [8]:
GGN_diag = FIM(layer_collection=lc,
               model=model,
               loader=loader,
               representation=PMatDiag,
               n_output=10,
               variant='classif_logits',
               device='cuda')

GGN_diag_inverse = GGN_diag.inverse()

diag_GGN_diag_inverse = GGN_diag_inverse.data
print(diag_GGN_diag_inverse.size())

GGNinv_diag_dw = GGN_diag_inverse.mv(dw)

torch.Size([12175])


# KFAC matrix

In [9]:
GGN_kfac = FIM(layer_collection=lc,
               model=model,
               loader=loader,
               representation=PMatKFAC,
               n_output=10,
               variant='classif_logits',
               device='cuda')

GGN_kfac_inverse = GGN_kfac.inverse()

diag_GGN_kfac_inverse = GGN_kfac_inverse.get_diag(split_weight_bias=True)
print(diag_GGN_kfac_inverse.size())

GGNinv_kfac_dw = GGN_kfac_inverse.mv(dw)

torch.Size([12175])
